<a href="https://colab.research.google.com/github/axiezai/nl-processors/blob/main/nl-processors/gaze/colab_classify_gaze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load in GazeCom labeled data for training `[x,y]` coordinates. 

In [ ]:
import requests, zipfile

# Download and unzip GazeCom training data in google colab:

fname = 'GazeCom.zip'
url = 'https://michaeldorr.de/smoothpursuit/GazeCom.zip'
r = requests.get(url, allow_redirects=True)

with open(fname, 'wb') as fd:
  fd.write(r.content)

with zipfile.ZipFile(fname, 'r') as zip_ref:
  zip_ref.extractall('/content/GazeCom_data')

In [2]:
import os
import natsort
import glob
import time
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Helper functions:    

In [3]:
def get_files(pattern):
    """
    Extracts file in alphanumerical order that match the provided pattern
    """
    if isinstance(pattern, list):
        pattern = os.path.join(*pattern)
        
    files = natsort.natsorted(glob.glob(pattern))
    if not files:
        raise FileNotFoundError('Pattern could not detect file(s)')
        
    return files


def set_seed(seed=None, seed_torch=True):
    if seed is None:
        seed = np.random.choice(2 ** 32)

    random.seed(seed)
    np.random.seed(seed)
    if seed_torch:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
    
    print(f'Random seed {seed} has been set.')


def seed_wworker(worker_id):
    """In case dataloader is used?
    """
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


def set_device():
    """Using GPU or CPU?
    """
    device = 'cuda' if torch.cuda.is_available() else "cpu"
    if device != 'cuda':
      print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
    else:
      print("GPU is enabled in this notebook.")

    return device

Use `scipy.io.arff` to load example file and convert to `DataFrame`:

In [4]:
raw_data = get_files('/content/GazeCom_data/gaze_arff/*/*.arff')
print(f'There are {len(raw_data)} raw eye gaze files')

labeled_data = get_files('/content/GazeCom_data/ground_truth/*/*.arff')
print(f'There are {len(labeled_data)} labeled eye gaze files')

# Load one in and examine what's inside ARFF files:
raw_arff = arff.loadarff(raw_data[0])
raw_df = pd.DataFrame(raw_arff[0])
print('Raw file:')
raw_df.head()

There are 844 raw eye gaze files
There are 844 labeled eye gaze files
Raw file:


,time,x,y,confidence
0,1000.0,590.9,5.2,1.0
1,5000.0,590.9,5.2,1.0
2,9000.0,590.6,5.0,1.0
3,13000.0,590.4,5.0,1.0
4,17000.0,589.8,5.2,1.0


In [5]:
labeled_arff = arff.loadarff(labeled_data[0])
labeled_df = pd.DataFrame(labeled_arff[0])
print('Labeled file:')
labeled_df.head()

Labeled file:


,time,x,y,confidence,handlabeller1,handlabeller2,handlabeller_final
0,1000.0,590.9,5.2,1.0,4.0,4.0,4.0
1,5000.0,590.9,5.2,1.0,4.0,4.0,4.0
2,9000.0,590.6,5.0,1.0,4.0,4.0,4.0
3,13000.0,590.4,5.0,1.0,4.0,4.0,4.0
4,17000.0,589.8,5.2,1.0,4.0,4.0,4.0


GazeCom data format: `timestamp in microseconds since start of movie (1e-6)` - `x posititon` - `y position` - `condifence value`. 

Labels (need to figure out ordering 1-4):


*   Fixation
*   Saccade
*   Smooth pursuit
*   noise



Acquisition details from paper:


*   250Hz sampling rate
*   Subjects were 45cm away from screen
*   Screen had 40cm width and 30cm height
*   Resolution is 1280x960 pixels
*   About 26.7 pixels on screen ~ 1 degree of visual angle



We hope to train a DL to classify the `<x,y>` positions.

`extract_windows` function copied over from: [Startsev 2018: Deep eye movement (EM) classifier: a 1D CNN-BLSTM model](https://github.com/MikhailStartsev/deep_em_classifier)

`izip_longest` changed  to `zip_longest` in Python3

In [93]:
def zip_equal(*args):
    """
    Iterate the zip-ed combination of @args, making sure they have the same length
    :param args: iterables to zip
    :return: yields what a usual zip would
    """
    fallback = object()
    for combination in itertools.zip_longest(*args, fillvalue=fallback):
        if any((c is fallback for c in combination)):
            raise ValueError('zip_equals arguments have different length')
        yield combination

def extract_windows(X, y, window_length,
                    padding_features=0,
                    downsample=1, temporal_padding=False):
    """
    Extract fixed-sized (@window_length) windows from arbitrary-length sequences (in X and y),
    padding them, if necessary (mirror-padding is used).
    :param X: input data; list of arrays, each shaped like (NUM_SAMPLES, NUM_FEATURES);
              each list item corresponds to one eye tracking recording (one observer & one stimulus clip)
    :param y: corresponding labels; list of arrays, each shaped like (NUM_SAMPLES,);
              each list element corresponds to sample-level eye movement class labels in the respective sequence;
              list elements in X and y are assumed to be matching.
    :param window_length: the length of resulting windows; this is the input "context size" in the paper, in samples.
    :param padding_features: how many extra samples to take in the feature (X) space on each side
                             (resulting X will have sequence length longer than resulting Y, by 2 * padding_features,
                             while Y will have sample length of @window_length);
                             this is necessary due to the use of valid padding in convolution operations in the model;
                             if all convolutions are of size 3, and if they all use valid padding, @padding_features
                             should be set to the number of convolution layers.
    :param downsample: take each @downsample'th window; if equal to @window_length, no overlap between windows;
                       by default, all possible windows with the shift of 1 sample between them will be created,
                       resulting in NUM_SAMPLES-1 overlap; if overlap of K samples is desired, should set
                       @downsample=(NUM_SAMPLES-K)
    :param temporal_padding: whether to pad the entire sequences, so that the first window is centered around the
                             first sample of the real sequence (i.e. the sequence of recorded eye tracking samples);
                             not used
    :return: two lists of numpy arrays:
                (1) a list of windows corresponding to input data (features),
                (2) a list of windows corresponding to labels we will predict.
                These can be used as input to network training procedures, for example.
    """
    res_X = []
    res_Y = []
    # iterate through each file in this subset of videos
    for x_item, y_item in zip_equal(X, y):
        # pad for all windows
        padding_size_x = padding_features
        padding_size_y = 0
        if temporal_padding:
            padding_size_x += window_length / 2
            padding_size_y += window_length / 2

        padded_x = np.pad(x_item, (padding_size_x, padding_size_x), 'reflect')
        # padded_y = np.pad(y_item, ((padding_size_y, padding_size_y), (0, 0)), 'reflect')
        padded_y = np.pad(y_item, (padding_size_y, padding_size_y), 'reflect')
        
        # Extract all valid windows in @padded_x, with given downsampling and size.
        # @res_X will have windows of size @window_length + 2*@padding_features
        window_length_x = window_length + 2 * padding_features
        res_X += [padded_x[i:i + window_length_x, :] for i in
                  range(0, padded_x.shape[0] - window_length_x + 1, downsample)]
        # @res_Y will have windows of size @window_length, central to the ones in @res_X
        res_Y += [padded_y[i:i + window_length] for i in
                  range(0, padded_y.shape[0] - window_length + 1, downsample)]
    return res_X, res_Y

In [94]:
coords = []
coords.append(labeled_df[['x', 'y']].values)
labels = []
bin_labels = labeled_df['handlabeller_final'].values
# bin_labels = pd.get_dummies(labeled_df['handlabeller_final']).values
labels.append(bin_labels)
x, y = extract_windows(coords, labels, 263, padding_features=3)

Create DataLoader for dataset:

In [99]:
# Create a custom dataset for GazeCom files:
from torch.utils.data import Dataset, DataLoader

class GazeComDataset(Dataset):
  def __init__(self, data_list, transforms = None):
    self.data_list = data_list
    self.df = pd.DataFrame(arff.loadarff(data_list[0])[0])
    # self.labels = pd.get_dummies(self.df['handlabeller_final']).values
    self.labels = self.df['handlabeller_final'].values
    self.transforms = transforms
    coords = []
    labels = []
    coords.append(self.df[['x', 'y']].values)
    labels.append(self.labels)
    self.window_x, self.window_y = extract_windows(coords, labels, window_length=129, padding_features=3)

  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, idx):
    # Return 1 sample and label according to idx:
    # print(idx)
    data = pd.DataFrame(arff.loadarff(self.data_list[idx])[0])
    xy_timeseries = data[['x', 'y']].values
    # bin_label = pd.get_dummies(data['handlabeller_final']).values
    bin_label = data['handlabeller_final'].values
    coords = []
    labels = []
    coords.append(xy_timeseries)
    labels.append(bin_labels)
    winx, winy = extract_windows(coords, labels, window_length=129, padding_features=3)
    rand_index = torch.randint(low=0, high=len(winy), size=(1,))
    return winx[rand_index], winy[rand_index]

In [100]:
dataset = GazeComDataset(labeled_data)

Try to iterate through a `DataLoader`:

In [101]:
testing_split = 0.2
batch_size = 4
shuffle = True

# create dataset:
dataset = GazeComDataset(labeled_data)
dataset_size = len(dataset)
print(f'Dataset has {dataset_size} .arff files')
indices = list(range(dataset_size))
split = int(np.floor(testing_split * dataset_size))

# DataLoader and split:

train_indices, test_indices = indices[split:], indices[:split]

train_loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)

# Iterate through DataLoader and view an example:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
# labels do not get padded!
print(f"Labels batch shape: {train_labels.size()}")

Dataset has 844 .arff files
113
713
368
602
Feature batch shape: torch.Size([4, 135, 8])
Labels batch shape: torch.Size([4, 129])


In [ ]:
DEVICE = set_device()
SEED = 42
set_seed(SEED)

Random seed 42 has been set.
